# 🎮 League of Legends Team Optimizer

This notebook provides an interactive interface for optimizing League of Legends team compositions using real player data from the Riot Games API.

## Features:
- 🔍 **Player Analysis**: Fetch champion mastery and performance data
- 🎯 **Team Optimization**: Find optimal role assignments
- 🏆 **Champion Recommendations**: Get champion suggestions for each role
- 📊 **Performance Analytics**: Analyze player strengths and synergies

---

## 🚀 Setup

Run this cell first to clone the repository and set up the environment.

In [ ]:
# Clone the repository
!git clone https://github.com/nicholas-golle/lol-team-optimizer.git
%cd lol-team-optimizer

# Install dependencies
!pip install -r requirements.txt

# Run setup
exec(open('colab_setup.py').read())

## 🔑 API Key Setup (Alternative)

If you didn't set up your API key in the previous step, you can do it here:

In [ ]:
import os
from getpass import getpass

# Securely input your API key
api_key = getpass("Enter your Riot API key (input will be hidden): ")

if api_key and api_key.startswith('RGAPI-'):
    os.environ['RIOT_API_KEY'] = api_key
    
    # Save to .env file
    with open('.env', 'w') as f:
        f.write(f'RIOT_API_KEY={api_key}\n')
    
    print("✅ API key configured successfully!")
else:
    print("⚠️ Invalid API key format. Expected: RGAPI-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx")
    print("Get your API key from: https://developer.riotgames.com/")

## 🎯 Quick Start

Test the system and run a quick demo:

In [ ]:
from colab_setup import quick_demo

# Run system test
cli = quick_demo()

if cli:
    print("\n🎮 System ready! You can now use the optimizer.")
else:
    print("\n❌ Setup failed. Please check your API key and try again.")

## 👥 Add Players

Add players to your team roster:

In [ ]:
from lol_team_optimizer.cli import CLI
from lol_team_optimizer.models import Player

# Initialize CLI
cli = CLI()

def add_player_interactive():
    """Interactive player addition for Colab."""
    name = input("Enter player name: ").strip()
    riot_id = input("Enter Riot ID (gameName#tagLine): ").strip()
    
    if not name or not riot_id or '#' not in riot_id:
        print("❌ Invalid input. Please provide both name and Riot ID.")
        return
    
    print(f"\n🔍 Validating {riot_id}...")
    
    try:
        # Validate Riot ID
        if cli.riot_client:
            summoner_data = cli.riot_client.get_summoner_data(riot_id)
            if summoner_data:
                puuid = summoner_data.get('puuid', '')
                print(f"✅ Found player: {riot_id}")
                
                # Create player with default preferences
                player = Player(
                    name=name,
                    summoner_name=riot_id,
                    puuid=puuid,
                    role_preferences={role: 3 for role in ["top", "jungle", "middle", "support", "bottom"]}
                )
                
                # Save player
                players = cli.data_manager.load_player_data()
                players.append(player)
                cli.data_manager.save_player_data(players)
                
                print(f"✅ Player '{name}' added successfully!")
                
                # Fetch API data
                fetch_data = input("Fetch champion mastery and performance data? (y/n): ").strip().lower()
                if fetch_data == 'y':
                    print("\n📊 Fetching player data...")
                    success = cli._fetch_player_api_data(player)
                    if success:
                        print("✅ Player data populated successfully!")
                    else:
                        print("⚠️ Some data could not be fetched.")
                
            else:
                print(f"❌ Riot ID '{riot_id}' not found.")
        else:
            print("❌ API not available. Cannot validate Riot ID.")
            
    except Exception as e:
        print(f"❌ Error adding player: {e}")

# Add a player
add_player_interactive()

## 📋 View Players

See all players in your roster:

In [ ]:
# Load and display players
players = cli.data_manager.load_player_data()

if players:
    print(f"📋 Current Players ({len(players)}):")
    print("=" * 50)
    
    for i, player in enumerate(players, 1):
        print(f"{i}. {player.name}")
        print(f"   Summoner: {player.summoner_name}")
        print(f"   Champions: {len(player.champion_masteries)}")
        print(f"   Performance Data: {len(player.performance_cache)} roles")
        print(f"   Last Updated: {player.last_updated.strftime('%Y-%m-%d %H:%M') if player.last_updated else 'Never'}")
        print()
else:
    print("📭 No players found. Add some players first!")

## 🎯 Team Optimization

Find the optimal team composition:

In [ ]:
def optimize_team():
    """Run team optimization and display results."""
    players = cli.data_manager.load_player_data()
    
    if len(players) < 5:
        print(f"❌ Need at least 5 players for optimization. Currently have {len(players)} players.")
        return
    
    print(f"🎯 Optimizing team composition for {len(players)} players...")
    
    try:
        # Run optimization
        result = cli.optimizer.optimize_team(players)
        
        print("\n" + "=" * 50)
        print("🏆 OPTIMAL TEAM COMPOSITION")
        print("=" * 50)
        
        best = result.best_assignment
        print(f"Total Score: {best.total_score:.2f}")
        print(f"Optimization Time: {result.optimization_time:.2f} seconds")
        print()
        
        # Display role assignments
        print("Role Assignments:")
        print("-" * 20)
        
        roles = ["top", "jungle", "middle", "support", "bottom"]
        for role in roles:
            player_name = best.assignments.get(role, "Unassigned")
            individual_score = best.individual_scores.get(player_name, 0)
            
            print(f"{role.upper():8} | {player_name:15} | Score: {individual_score:.2f}")
            
            # Display champion recommendations
            if role in best.champion_recommendations and best.champion_recommendations[role]:
                print(f"         | Recommended Champions:")
                for i, rec in enumerate(best.champion_recommendations[role][:3], 1):
                    suitability_desc = "Perfect" if rec.role_suitability >= 0.9 else "Excellent" if rec.role_suitability >= 0.7 else "Good"
                    print(f"         |   {i}. {rec.champion_name}")
                    print(f"         |      Level {rec.mastery_level} | {rec.mastery_points:,} pts | {suitability_desc}")
            else:
                print(f"         | No champion recommendations available")
            print()
        
        # Show synergies
        if best.synergy_scores:
            print("Key Synergies:")
            print("-" * 15)
            sorted_synergies = sorted(best.synergy_scores.items(), key=lambda x: x[1], reverse=True)
            for (player1, player2), synergy in sorted_synergies[:3]:
                print(f"  {player1} + {player2}: {synergy:.2f}")
            print()
        
        return result
        
    except Exception as e:
        print(f"❌ Optimization failed: {e}")
        return None

# Run optimization
optimization_result = optimize_team()

## 📊 Player Analysis

Analyze individual player data:

In [ ]:
def analyze_player():
    """Analyze a specific player's data."""
    players = cli.data_manager.load_player_data()
    
    if not players:
        print("📭 No players found.")
        return
    
    print("Select player to analyze:")
    for i, player in enumerate(players, 1):
        print(f"{i}. {player.name} ({player.summoner_name})")
    
    try:
        choice = int(input("\nEnter player number: ")) - 1
        if 0 <= choice < len(players):
            player = players[choice]
            
            print(f"\n" + "=" * 40)
            print(f"📊 PLAYER ANALYSIS: {player.name.upper()}")
            print("=" * 40)
            
            print(f"Name: {player.name}")
            print(f"Summoner: {player.summoner_name}")
            print(f"Last Updated: {player.last_updated.strftime('%Y-%m-%d %H:%M') if player.last_updated else 'Never'}")
            
            # Role preferences
            print(f"\nRole Preferences:")
            print("-" * 20)
            sorted_prefs = sorted(player.role_preferences.items(), key=lambda x: x[1], reverse=True)
            for role, pref in sorted_prefs:
                stars = "★" * pref + "☆" * (5 - pref)
                print(f"{role.capitalize():8} | {stars} ({pref}/5)")
            
            # Champion mastery
            if player.champion_masteries:
                print(f"\nTop Champions:")
                print("-" * 15)
                top_champs = sorted(player.champion_masteries.values(), key=lambda x: x.mastery_points, reverse=True)[:5]
                for i, champ in enumerate(top_champs, 1):
                    roles_str = ", ".join(champ.primary_roles) if champ.primary_roles else "Flexible"
                    print(f"{i}. {champ.champion_name} ({roles_str})")
                    print(f"   Level {champ.mastery_level} | {champ.mastery_points:,} points")
            
            # Performance data
            if player.performance_cache:
                print(f"\nPerformance Data:")
                print("-" * 18)
                for role, perf in player.performance_cache.items():
                    if isinstance(perf, dict):
                        wr = perf.get('win_rate', 0) * 100
                        matches = perf.get('matches_played', 0)
                        kda = perf.get('avg_kda', 0)
                        print(f"{role.capitalize()}: {wr:.1f}% WR over {matches} games (KDA: {kda:.1f})")
            
        else:
            print("❌ Invalid selection.")
            
    except (ValueError, IndexError):
        print("❌ Invalid input.")

# Analyze a player
analyze_player()

## 🖥️ Full CLI Interface

Run the complete CLI interface (note: this will be interactive):

In [ ]:
# WARNING: This will start the full interactive CLI
# It may not work well in Colab due to input limitations
# Use the individual cells above for better Colab experience

print("⚠️ Starting full CLI interface...")
print("Note: This may not work well in Colab. Use individual cells above for better experience.")
print("Press Ctrl+C to interrupt if needed.")

# Uncomment the next line to run the full CLI
# cli.main()

## 💡 Tips for Google Colab

1. **API Key Security**: Your API key is stored in the session and `.env` file. It will be lost when the runtime restarts.

2. **Data Persistence**: Player data is saved to the `data/` directory. Download it if you want to keep it.

3. **Rate Limits**: Be mindful of API rate limits (120 requests per 2 minutes for development keys).

4. **Interactive Elements**: Some CLI features may not work perfectly in Colab. Use the individual cells above.

5. **Restart Runtime**: If you encounter issues, try restarting the runtime and running the setup cell again.

---

**Need help?** Check the [API Setup Guide](https://github.com/YOUR_USERNAME/lol-team-optimizer/blob/main/API_SETUP.md) for detailed instructions.